# Trip by Trip Optimization

In [ ]:
from pathlib import Path
import time
import pandas as pd
import pickle
from stochopy.optimize import minimize
import datetime

import sys
path = Path.cwd().parent.parent
sys.path.insert(0,str(path))
import file_structure_setup
config = file_structure_setup.filepaths(path)

import impedance_calibration.src.stochastic_optimization as stochastic_optimization

# Edit this block

In [ ]:
# determine the ouput name of the calibration outputs
calibration_name = 'calibration8'

# determine variables, impedance type, and search range
betas_tup = (
    {'col':'2lpd','type':'link','range':[0,3]},
    {'col':'3+lpd','type':'link','range':[0,3]},
    {'col':'(30,inf) mph','type':'link','range':[0,3]},
    # {'col':'(40,inf) mph','type':'link','range':[0,3]},
    # {'col':'[4k,10k) aadt','type':'link','range':[0,3]},
    {'col':'[10k,inf) aadt','type':'link','range':[0,3]},
    # {'col':'[4,6) grade','type':'link','range':[0,3]},
    {'col':'[6,inf) grade','type':'link','range':[0,3]},
    {'col':'bike lane','type':'link','range':[-1,0]},
    # {'col':'cycletrack','type':'link','range':[-1,0]},
    {'col':'multi use path and cycletrack','type':'link','range':[-1,0]},
    {'col':'unsig_major_road_crossing','type':'turn','range':[0,2]}
)
# determine the objective function to use and other settings
objective_function = stochastic_optimization.buffer_overlap
batching = False
stochastic_optimization_settings = {
    'method':'pso',
    'options': {'maxiter':100,'popsize':3}
}

# Just let this block run

In [ ]:
links, turns, length_dict, geo_dict, turn_G = stochastic_optimization.import_calibration_network(config)

with (config['calibration_fp']/'ready_for_calibration.pkl').open('rb') as fh:
    full_set = pickle.load(fh)

args = (
    [], # empty list for storing past calibration results
    betas_tup, # tuple containing the impedance spec
    stochastic_optimization.match_results_to_ods(full_set), # list of OD network node pairs needed for shortest path routing
    full_set, # dict containing the origin/dest node and map matched edges
    stochastic_optimization.link_impedance_function, # link impedance function to use
    "travel_time_min", # column with the base the base impedance in travel time or distance
    stochastic_optimization.turn_impedance_function, # turn impedance function to use
    links,turns,turn_G, # network parts
    objective_function, # loss function to use
    {'length_dict':length_dict,'geo_dict':geo_dict},#,'trace_dict':traces}, # keyword arguments for loss function
    True, #whether to print the results of each iteration
    True, #whether to store calibration results
    batching # whether to batch results to help speed up computation time, if yes input the number to batch with
)

start = time.time()
print([x['col'] for x in betas_tup]+['objective_function'])
x = minimize(stochastic_optimization.impedance_calibration,
             stochastic_optimization.extract_bounds(betas_tup),
             args=args,
             **stochastic_optimization_settings)
end = time.time()
print(f"Took {str(pd.Timedelta(seconds=end-start).round('s'))} hours")
print(f"{args[10].__name__}: {x.fun}")
print(x)

calibration_result = {
    'betas_tup': tuple({**item,'beta':x.x[idx].round(4)} for idx,item in enumerate(betas_tup)), # contains the betas
    'settings': stochastic_optimization_settings, # contains the optimization settings
    'objective_function': args[10].__name__, # objective function used
    'results': x, # stochastic optimization outputs
    'trips_calibrated': set(full_set.keys()), # saves which trips were calibrated
    'past_vals': args[0], # all of the past values/guesses
    'runtime': pd.Timedelta(end-start),
    'time': datetime.datetime.now()
}
#export but don't overwrite
export_fp = config['calibration_fp'] / f'calibration_results/{calibration_name}.pkl'
with stochastic_optimization.uniquify(export_fp).open('wb') as fh:
        pickle.dump(calibration_result,fh)